<a href="https://colab.research.google.com/github/vladcioaba/vault/blob/main/RoBERTa_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!git clone https://github.com/nishkalavallabhi/OneStopEnglishCorpus.git

fatal: destination path 'OneStopEnglishCorpus' already exists and is not an empty directory.


In [15]:
!wget -nc https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py
!wget -nc https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_training_args.py

File ‘seq2seq_trainer.py’ already there; not retrieving.

File ‘seq2seq_training_args.py’ already there; not retrieving.



In [16]:
!pip install datasets==1.0.2
!pip install transformers
!pip install rouge_score

In [17]:
# Set correct tensorflow
%tensorflow_version 2.x

In [18]:
import datasets
import transformers
import pandas as pd
from datasets import Dataset

#Tokenizer
from transformers import RobertaTokenizerFast

#Encoder-Decoder Model
from transformers import EncoderDecoderModel

#Training
from seq2seq_trainer import Seq2SeqTrainer
from seq2seq_training_args import Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

In [19]:
import io

data = []
all_files = ['OneStopEnglishCorpus/Sentence-Aligned/ADV-ELE.txt']

for filename in all_files:
  with io.open(filename, 'rb') as f:
    while True:
      line1 = f.readline().decode()
      line2 = f.readline().decode()
      if not line1 or not line2:
        break
      f.readline() # spacer
      data.append([line1, line2])

df = pd.DataFrame(data, columns = ['Summary', 'Text'])
df = df.dropna()
len_ = len(df)
print("Data size:", len_)
df.head()

from datasets import Dataset
train_data=Dataset.from_pandas(df[:len_ - 20])
val_data=Dataset.from_pandas(df[len_ - 20:len_ - 5])
test_data=Dataset.from_pandas(df[-5:])

print("Train size:", len(train_data))
print("Val   size:", len(val_data))
print("Test  size:", len(test_data))

Data size: 2166
Train size: 2146
Val   size: 15
Test  size: 5


In [20]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
#parameter setting
batch_size=256  #
encoder_max_length=40
decoder_max_length=8

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  print(batch)
  inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["Summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because RoBERTa automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

#processing training data
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
   #remove_columns=["Text", "Summary"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

#processing validation data
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


OrderedDict([('Summary', ['The Seattle-based company has applied for its brand to be a top-level domain name (currently .com), but the South American governments argue this would prevent the use of this internet address for environmental protection, the promotion of indigenous rights and other public interest uses.\n', 'Until now, the differences between commercial, governmental and other types of identity were easily distinguished in every internet address by .com, .gov and 20 other categories.\n']), ('Text', ['Amazon has asked for its company name to be a top-level domain name (currently .com), but the South American governments say this would stop the use of this internet address for environmental protection, indigenous rights and other public interest uses.\n', 'Until now, the differences between commercial, governmental and other types of identity were easy to see in every internet address by the use of .com, .gov and 20 other categories.\n']), ('__index_level_0__', [0, 1])])


OrderedDict([('Summary', ['The Seattle-based company has applied for its brand to be a top-level domain name (currently .com), but the South American governments argue this would prevent the use of this internet address for environmental protection, the promotion of indigenous rights and other public interest uses.\n', 'Until now, the differences between commercial, governmental and other types of identity were easily distinguished in every internet address by .com, .gov and 20 other categories.\n', 'Amazon has applied for dozens of new domains, including .shop, .song, .book and .kindle.\n', 'Allowing private companies to register geographical names as gTLDs to reinforce their brand strategy or to profit from the meaning of these names does not serve, in our view, the public interest, the Brazilian Ministry of Science and Technology said.\n', 'Brazil said its views were endorsed last month by other members of the Amazon Cooperation Treaty (Bolivia, Colombia, Ecuador, Guyana, Suriname a

OrderedDict([('Summary', ['Ethical hacker Typical salary: 60,000 to 90,000 at team- leader level, while a newly quali ed hacker can expect a minimum salary of 35,000 to 50,000.\n', 'What the job involves: A company will pay an ethical hacker to hack into its computer system to see how well it might stand up to a real attack.\n', 'Quali cations: You dont necessarily need a degree in computer science.\n', 'The industry accepts individuals with a very wide range of academic quali cations and skills.\n', 'Quali cations: To dive offshore, you must have diving-inspection and medical-technician quali cations.\n', 'To be able to dispose of the bombs safely, youll also need an explosive ordnance disposal quali cation and years of experience.\n', 'To succeed as a bomb-disposal diver, you need ... to stay calm in stressful situations.\n', 'Worst thing about the job: Expect to be away from home at least six months of the year.\n', 'You also need a strong sense of personal ethics and an understandi

In [21]:
from transformers import EncoderDecoderModel
roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("roberta-base", "roberta-base", tie_encoder_decoder=True)

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.0.crossattention.self.query.weight', 'roberta.encoder.layer.0.crossattention.self.query.bias', 'roberta.encoder.layer.0.crossattention.self.key.weight', 'roberta.encoder.layer.0.crossattention.self.key.bias', 'roberta.encoder.layer.0.crossattention.self.value.weight', 'roberta.encoder.layer.0.crossattention.self.value.bias', 'roberta.encoder.layer.0.crossattention.output.dense.weight', 'roberta.encoder.layer.0.crossattention.output.dense.bias', 'roberta.encoder.layer.0.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.0.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.1.crossattention.self.query.weight', 'roberta.encoder.layer.1.crossattention.self.query.bias', 'roberta.encoder.layer.1.crossattention.self.key.weight', 'roberta.encoder.layer.1.crossattention.self.key.bias', 'roberta.encoder.layer.1.crossattention.

In [22]:
# set special tokens
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
roberta_shared.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
roberta_shared.config.max_length = 40
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 3
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 4
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size

In [23]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [24]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./checkpoint-16",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    do_train=True,
    do_eval=True,
    logging_steps=2, 
    save_steps=16, 
    eval_steps=500, 
    warmup_steps=500, 
    overwrite_output_dir=True,
    save_total_limit=1,
    fp16=True,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=roberta_shared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

The `config.pad_token_id` is `None`. Using `config.eos_token_id` = 2 for padding..
/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:835: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the le

Step,Training Loss
2,10.640200
4,10.505700
6,10.607200
8,10.498100
10,10.426800
12,10.405400
14,10.192700
16,9.923700
18,9.794800
20,9.561200


TrainOutput(global_step=27, training_loss=9.950665473937988, metrics={'train_runtime': 45.887, 'train_samples_per_second': 0.588, 'total_flos': 284898260843712, 'epoch': 3.0})

In [25]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
model = EncoderDecoderModel.from_pretrained("./checkpoint-16")
model.to("cuda")
batch_size = 256

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=40, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch
results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["Text"])
pred_str = results["pred"]
label_str = results["Summary"]

The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [26]:
for l1, l2 in zip(label_str, pred_str):
  print(l1.strip())
  print(l2.strip())
  print('*' * 16)

The charity National Trust, which employs many of its seasonal workers on zero-hours contracts, said it offered the same pay and bene ts to those workers, pro rata, as full-time staff, but needed some workers to be on a more  exible arrangement.
sss hess thess andss,ss thatss ofss Hess itss whossHessingsss himss
****************
Our properties have told us its important to be able to reorganize staff rotas quickly to respond to the weather and zero-hours contracts allow us to do this.
sss andss ofss,ss hess)ss thingss thess itss littless ass stillss inss
****************
While some employees welcome the  exibility of such contracts, for many, zero-hours contracts leave them insecure and unsure of when work will come, he said.
sss hess andss,ss ofssingsss thess itss'sssSss tossingss)ss
****************
The institutes  gures also suggest that 17% of employers in the private sector made use of zero- hours contracts, considerably lower than the 34% of organizations in the voluntary sector 

In [27]:
print("ROUGE 1 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid)
print("ROUGE 2 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid)
print("ROUGE L SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid)

ROUGE 1 SCORE:  Score(precision=0.0, recall=0.0, fmeasure=0.0)
ROUGE 2 SCORE:  Score(precision=0.0, recall=0.0, fmeasure=0.0)
ROUGE L SCORE:  Score(precision=0.0, recall=0.0, fmeasure=0.0)
